# Part 1: Matrix Multiplication Software Implementation

## 1. Overview

Matrix multiplication is one of the most fundamental operations in scientific computing. It represents the composition of linear mappings, symbolizing spatial transformations and rotations. This operation finds extensive applications in various fields, such as encryption and decryption in cryptography, simulation of input-output models in mathematical modeling, and serving as an essential computational tool for advanced algorithms. Therefore, accelerating the computation of matrix multiplication is a crucial problem.

In the previous FIR chapter, we focused on explaining the design philosophy of hardware optimization, providing a preliminary understanding of the emphasis in hardware design. In this chapter, we take it a step further, demonstrating how to design an efficient matrix multiplication accelerator by improving computational structures, optimizing data access, and enhancing parallelism. The goal is to enhance the computation speed for matrices of size 128*128 or even larger. We will compare the speed with the matrix multiplication operation in the Python Numpy library, visibly boosting the speed from 0.0571 seconds in software to 0.0021 seconds (block matrix architecture) and 0.0118 seconds (pulse array architecture) in hardware, achieving nearly 20 times and 5 times faster, respectively.

<img src="./image/Impl_of_Mmult_in_SW_and_HW.png" alt="Figure 1: Illustration of Matrix Multiplication Process"  style="zoom:70%;" />

## Matrix Multiplication Algorithm Description

Let's start by reviewing the matrix multiplication process, analyzing the algorithm is our first step in the acceleration design.

$$
\begin{bmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33}
\end{bmatrix}
\times
\begin{bmatrix}
b_{11} & b_{12} & b_{13} \\
b_{21} & b_{22} & b_{23} \\
b_{31} & b_{32} & b_{33}
\end{bmatrix}
=
\begin{bmatrix}
a_{11}b_{11} + a_{12}b_{21} + a_{13}b_{31} & a_{11}b_{12} + a_{12}b_{22} + a_{13}b_{32} & a_{11}b_{13} + a_{12}b_{23} + a_{13}b_{33} \\
a_{21}b_{11} + a_{22}b_{21} + a_{23}b_{31} & a_{21}b_{12} + a_{22}b_{22} + a_{23}b_{32} & a_{21}b_{13} + a_{22}b_{23} + a_{23}b_{33} \\
a_{31}b_{11} + a_{32}b_{21} + a_{33}b_{31} & a_{31}b_{12} + a_{32}b_{22} + a_{33}b_{32} & a_{31}b_{13} + a_{32}b_{23} + a_{33}b_{33} \\
\end{bmatrix}
$$

$$

\begin{bmatrix}
a_{11} & \cdots & a_{1n} \\
\vdots & \ddots & \vdots \\
a_{n1} & \cdots & a_{nn}
\end{bmatrix}
\times
\begin{bmatrix}
b_{11} & \cdots & b_{1n} \\
\vdots & \ddots & \vdots \\
b_{n1} & \cdots & b_{nn}
\end{bmatrix}
=
\begin{bmatrix}
a_{11}b_{11} + \cdots + a_{1n}b_{n1} & \cdots & a_{11}b_{1n} + \cdots + a_{1n}b_{nn} \\
\vdots & \ddots & \vdots \\
a_{n1}b_{11} + \cdots + a_{nn}b_{n1} & \cdots & a_{n1}b_{1n} + \cdots + a_{nn}b_{nn}
\end{bmatrix}

$$

From this process, we can deduce that the time complexity of matrix multiplication is $O(N^{3})$, indicating that obtaining a total of $N*N$ elements requires N multiplications and (N-1) additions for each element. Simultaneously, we need to access the $N^{2}$ elements of matrix B continuously N times. We know that we need to access data from the CPU and then perform calculations on the FPGA. Such data access is likely to slow down the computation progress.

As the matrix dimensions increase, the consumption of computational resources and data access becomes enormous. Calculating matrix C requires a total of $N^{3}$ multiplications, $N^{2}(N - 1)$ additions, and accessing data $2N^{3}$ times. For N=128, this translates to 2,097,152 multiplications, 2,080,768 additions, and accessing data 4,194,304 times. Therefore, accelerating matrix multiplication is crucial given the immense computational time involved.

In this case, we assume that the matrix dimensions to be accelerated are 128*128, and each element's value is an integer between 0 and 255.

## 2. Matrix Multiplication in Numpy

In Numpy, matrix multiplication can be easily achieved using the `np.dot()` function. In fact, this is already a fast implementation on the CPU because Numpy's matrix multiplication relies on a highly optimized matrix operation library based on C/C++. Additionally, Numpy automatically performs acceleration optimizations based on certain characteristics of the input matrices A and B.

In some special optimizations, its time complexity can be reduced from O(N^3) to O(N^2.3728639), a result simplified by François Le Gall using the Stanford method in the fall of 2014.

Below is a simple Python expression for multiplying two 2*2 matrices:

In [1]:
import numpy as np

A = np.array([[1,2],[3,4]])
B = np.array([[1,2],[3,4]])
C = np.dot(A, B)
print(C)

[[ 7 10]
 [15 22]]


To evaluate the computational speed, we can design a testing script to perform 10,000 iterations of 128*128 matrix multiplication calculations.

Here is an example of how you might structure the testing script in Python:

In [2]:
import random
import numpy as np

in1 = np.random.randint(255, size=(1, 16384))
in2 = np.random.randint(255, size=(1, 16384))
in1_py = np.resize(in1, (128, 128))
in2_py = np.resize(in2, (128, 128))

# Record the time for 10,000 multiplications
import time
start = time.time()
for i in range(0, 9999):
    out_py = np.dot(in1_py, in2_py)
end = time.time()

print("Time taken for one 128*128 matrix multiplication: {}s".format((end - start) / 10000))


Time taken for one 128*128 matrix multiplication: 0.0032365356683731077s


Hence, we can conclude that the time taken to perform 128*128 matrix multiplication in Numpy is approximately 0.00231 seconds.

---------------------------------------
<p align="center">Copyright&copy; 2024 Advanced Micro Devices</p>